In [1]:
from torch.nn.functional import softmax
from transformers import pipeline, GPT2TokenizerFast, GPT2LMHeadModel, AutoTokenizer, BertForMaskedLM
from autocorrect import Speller
from nltk.stem import WordNetLemmatizer, PorterStemmer
from textblob import TextBlob
from difflib import SequenceMatcher
from string import punctuation
from time import time
import numpy as np
import pandas as pd
import torch
import nltk
nltk.download('words')

[nltk_data] Downloading package words to C:\Users\bills-fish-
[nltk_data]     shack\AppData\Roaming\nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [2]:
topk = 200  # number of top predicted tokens to retrieve (before excluding non-words) 

class GPT2:
    def __init__(self, model="gpt2"):
        self.model     =   GPT2LMHeadModel.from_pretrained(model)
        self.tokenizer = GPT2TokenizerFast.from_pretrained(model)
        self.model_id  = model
    
    def get_word_probs(self, sentence, n=topk):  # adapted from raul on stackoverflow
        inputs = self.tokenizer.encode(sentence, return_tensors="pt")
        with torch.no_grad():
            outputs = self.model(inputs)
            predictions = outputs[0]
        candidates = predictions[0, -1, :]                          # Get the next token candidates.
        topk_i = torch.topk(candidates, n).indices.tolist()         # Get the top k next token candidates.
        all_probs = torch.nn.functional.softmax(candidates, dim=-1) # Get the token probabilities for all candidates.
        topk_probs = all_probs[topk_i].tolist()                     # Filter the token probabilities for the top k candidates.
        topk_tokens = [self.tokenizer.decode([idx]).strip()         # Decode the top k candidates back to words.
                       for idx in topk_i]
        return np.array(list(zip(topk_tokens, topk_probs)))

class BERT:
    def __init__(self, model="google-bert/bert-base-uncased"):
        self.model     = BertForMaskedLM.from_pretrained(model)
        self.tokenizer =   AutoTokenizer.from_pretrained(model)
        self.model_id  = model
        
    def get_word_probs(self, prompt, topk=topk):                  # Get topk masked token candidates
        inputs = self.tokenizer(prompt, return_tensors="pt")
        with torch.no_grad():
            logits = self.model(**inputs).logits
        mask_index  = (inputs.input_ids == self.tokenizer.mask_token_id)[0].nonzero(as_tuple=True)[0]
        mask_logits = logits.squeeze()[mask_index].squeeze()
        probs = softmax(mask_logits, dim=-1)
        topk_probs, topk_i = torch.topk(probs, topk, dim=-1)
        topk_tokens = np.array([self.tokenizer.decode([i]) for i in topk_i])
        return np.hstack((topk_tokens.reshape(-1,1), np.array(topk_probs).reshape(-1,1)))

M_GPT2 = GPT2("gpt2")
M_BERT = BERT("google-bert/bert-base-uncased")

BertForMaskedLM has generative capabilities, as `prepare_inputs_for_generation` is explicitly overwritten. However, it doesn't directly inherit from `GenerationMixin`. From 👉v4.50👈 onwards, `PreTrainedModel` will NOT inherit from `GenerationMixin`, and this model will lose the ability to call `generate` and other related functions.
  - If you're using `trust_remote_code=True`, you can get rid of this warning by loading the model with an auto class. See https://huggingface.co/docs/transformers/en/model_doc/auto#auto-classes
  - If you are the owner of the model architecture code, please modify your model class such that it inherits from `GenerationMixin` (after `PreTrainedModel`, otherwise you'll get an exception).
  - If you are not the owner of the model architecture class, please contact the model code owner to update it.
Some weights of the model checkpoint at google-bert/bert-base-uncased were not used when initializing BertForMaskedLM: ['bert.pooler.dense.bias', 'bert.pooler.dense

In [3]:
def similar(a, b):
    common_len = np.ceil((len(a)+len(b))/2)
    adjustment = 0
    adjustment_table = {1: 0.5, 2: 0.3, 3: 0.2, 4: 0.1}
    if common_len in adjustment_table: adjustment = adjustment_table[common_len]*(np.e**(-k*(np.abs(len(a)-len(b))-ap))-bp)
    return SequenceMatcher(None, a, b).ratio() + adjustment
def rreplace(string, word, new_word):
    start = string.rfind(word)
    return string[0:start] + new_word + string[start+len(word):]
def blob_correct(sentence):
    blob = TextBlob(sentence)
    return str(blob.correct())
    
threshold    = {"constant":    lambda n: base_t,
                "linear":      lambda n: base_t + (base_t * (n-1)),
                "exponential": lambda n: base_t * (n**threshold_e),
                "jump-exp":    lambda n: base_t * (max(n-1,1)**threshold_e),        # jump thresholds start growing after n=2
                "jump-lin":    lambda n: base_t + (base_t * max(n-2, 0))}
lemmatizer  = WordNetLemmatizer()
lemma       = lambda x: lemmatizer.lemmatize(x)
stemmer     = PorterStemmer()
stem        = lambda x: stemmer.stem(x)
spell       = Speller()
wl          = set(nltk.corpus.words.words())
log_map     = lambda e: np.vectorize(lambda x: np.power(np.log(x/0.5)/np.log(2), e))  # specify exponent to return vectorized mapping
after_slash = lambda x: x[(x.rfind("/")+1 if x.rfind("/") != -1 else 0):]

In [4]:
def get_probs(string, back_n):
    probs = []
    places = range(1,back_n+1)
    string = string.strip()
    words  = string.split()
    last_space = string.rfind(' ')
    for n in places:
        if n > len(words) or len(words) == 1: break
        spelled = False
        if n > 1:
            model  = M_BERT
            masked = "[MASK]" + words[-n][-1] if not words[-n][-1].isalpha() else "[MASK]"
            target = words[-n].strip(punctuation)
            prompt = ' '.join(words[:-n] + [masked] + words[len(words)-(n-1):])
        else:
            model  = M_GPT2
            string = string.strip()
            last_space = string.rfind(' ')
            prompt = string[:last_space]
        prob = model.get_word_probs(prompt)
        if len(prob) > 0:
            probs.append(prob)
        else:
            probs.append(np.array([['dummy', 0]]))
    return probs
def correction(string, back_n, pprobs=False):
    places = reversed(range(1,back_n+1))
    if back_n == 0: places = [1, 3, 2]
    string = string.strip()
    words  = string.split()
    last_space = string.rfind(' ')
    for n in places:
        if n > len(words) or len(words) == 1: break
        spelled = False
        if n > 1:
            model  = M_BERT
            masked = "[MASK]" + words[-n][-1] if not words[-n][-1].isalpha() else "[MASK]"
            target = words[-n].strip(punctuation)
            prompt = ' '.join(words[:-n] + [masked] + words[len(words)-(n-1):])
            target = words[-n].strip(punctuation)
            if target != spell(target):
                spelled = True
        else:
            model  = M_GPT2
            string = string.strip()
            last_space = string.rfind(' ')
            prompt = string[:last_space]
            target = string[last_space+1:].strip(punctuation)
            target = words[-n].strip(punctuation)
            if target != spell(target):
                spelled = True
                
        if pprobs:
            probs = pprobs[n-1]
        else:
            probs = model.get_word_probs(prompt)
            
        probs[:, 1] = probs[:, 1].astype(float) / probs[:, 1].astype(float).sum()
        probsp = [(str(word), float(prob), float(similar(target.lower(), word.lower()))) 
                  for word, prob in probs if word in wl]
        close_probs = [prob for prob in probsp if prob[2] > 0.5 and prob[1] >= 0.001]
        props = [(word, (prob**prob_exp)*log_map(log_exp)(sim)) for word, prob, sim in close_probs]
        props = sorted(props, reverse=True, key=lambda x: x[1])
        
        probN = threshold[threshold_t](n)
        make_correction = False
        if len(props) > 0 and float(props[0][1]) > probN:
            make_correction = True
            irr_t = float(props[0][1]) * relevency_t
            for word, score in props: 
                if float(score) < irr_t: break
                elif target.lower() == word.lower() or stem(target.lower()) == word.lower() or lemma(target.lower()) == word.lower():
                    make_correction = False
        if make_correction: return (n, props[0][0], spelled)
        if spelled: 
            target = spell(target)
            
    return False
def process_correction(string, back_n, pprobs=False):
    corrected = string
    words     = string.split()
    is_correction = correction(string, back_n, pprobs)
    if is_correction:
        n, word, _ = is_correction
        words[-n] = word if words[-n][-1] not in punctuation else word + words[-n][-1]
        corrected = " ".join(words)
    return corrected if corrected != string else False

In [5]:
strings = pd.read_csv("strings.txt", quotechar='"', header=None, index_col=False, skipinitialspace=True).values
strings[:5], len(strings)

(array([['when you come in can you remember to feel the cat?',
         'when you come in can you remember to feed the cat?'],
        ['when you come in can you remember to feed the cad?',
         'when you come in can you remember to feed the cat?'],
        ['when you come in can you remember to feed the cad? He',
         'when you come in can you remember to feed the cat? He'],
        ['when you come in can you remember to feed the cad? He needs',
         'when you come in can you remember to feed the cat? He needs'],
        ['can you remember to feed the cad when',
         'can you remember to feed the cat when']], dtype=object),
 295)

In [6]:
sb_size = 250
super_batch = strings[np.random.choice(strings.shape[0], sb_size, replace=False),:]
probs = []
times = []
for i, string in enumerate(super_batch):
    t0 = time()
    probs.append(get_probs(string[0], 4))
    times.append(time()-t0)
    print(f'({i+1}/{len(super_batch)}): {times[-1]:.3f} ', end='')
print(f'\naverage: {np.mean(times):.3f} seconds')

(1/250): 0.329 (2/250): 0.126 (3/250): 0.131 (4/250): 0.133 (5/250): 0.126 (6/250): 0.135 (7/250): 0.130 (8/250): 0.134 (9/250): 0.147 (10/250): 0.133 (11/250): 0.000 (12/250): 0.134 (13/250): 0.123 (14/250): 0.160 (15/250): 0.136 (16/250): 0.163 (17/250): 0.181 (18/250): 0.073 (19/250): 0.150 (20/250): 0.155 (21/250): 0.134 (22/250): 0.142 (23/250): 0.070 (24/250): 0.185 (25/250): 0.161 (26/250): 0.147 (27/250): 0.385 (28/250): 0.369 (29/250): 0.350 (30/250): 0.185 (31/250): 0.302 (32/250): 0.294 (33/250): 0.201 (34/250): 0.305 (35/250): 0.368 (36/250): 0.244 (37/250): 0.122 (38/250): 0.315 (39/250): 0.328 (40/250): 0.328 (41/250): 0.393 (42/250): 0.391 (43/250): 0.439 (44/250): 0.349 (45/250): 0.331 (46/250): 0.375 (47/250): 0.349 (48/250): 0.414 (49/250): 0.450 (50/250): 0.188 (51/250): 0.377 (52/250): 0.382 (53/250): 0.341 (54/250): 0.257 (55/250): 0.295 (56/250): 0.353 (57/250): 0.000 (58/250): 0.257 (59/250): 0.121 (60/250): 0.339 (61/250): 0.346 (62/250): 0.373 (63/250): 0.299 (

In [7]:
back_n  = 4  # number of words back from end of string, 1 is just last word, 0 is [1, 3, 2]
k       = 1.2  # exponent parameter for exponential decay of word length augmedented SequenceMatcher
ap      = 0.55  # exponent parameter
bp      = 1  # exponent parameter
log_exp      = 5  # exponent parameter for logarithmic mapping
prob_exp     = 1  # raise probability to power in ((prob**power)*log-sim)
consider_top = 100  # max top model word predictions considered
relevency_t  = 0.07  # threshold defined by portion of top proposition to exclude much smaller scored propositions for correcting
base_t       = 0.0015   # decision threshold for last word: base threshold
threshold_e  = 1.8  # exponent for exponential thresholds
threshold_t  = "exponential"  # function defines decision threshold for word n from end

batch_size = 100
batch_i = np.random.choice(super_batch.shape[0], batch_size, replace=False)
batch = super_batch[batch_i,:]

corrections = []
times = []
for i, (pprobs, (x, _)) in enumerate(zip([probs[i] for i in batch_i], batch)): 
    t0 = time()
    corrections.append(process_correction(x, back_n, pprobs))
    times.append(time()-t0)
    print(f'({i+1}/{len(batch)}): {times[-1]:.3f} ', end='')
print(f'\naverage: {np.mean(times):.3f} seconds')

(1/100): 0.000 (2/100): 2.545 (3/100): 0.009 (4/100): 0.011 (5/100): 0.004 (6/100): 0.008 (7/100): 0.005 (8/100): 0.010 (9/100): 0.000 (10/100): 0.011 (11/100): 0.016 (12/100): 0.006 (13/100): 0.006 (14/100): 0.000 (15/100): 0.007 (16/100): 0.005 (17/100): 0.090 (18/100): 0.010 (19/100): 0.002 (20/100): 0.006 (21/100): 0.007 (22/100): 0.000 (23/100): 0.007 (24/100): 0.007 (25/100): 0.008 (26/100): 0.096 (27/100): 0.011 (28/100): 0.004 (29/100): 0.009 (30/100): 0.009 (31/100): 0.008 (32/100): 0.007 (33/100): 0.000 (34/100): 0.006 (35/100): 0.000 (36/100): 0.000 (37/100): 0.002 (38/100): 0.000 (39/100): 0.004 (40/100): 0.009 (41/100): 0.000 (42/100): 0.003 (43/100): 0.009 (44/100): 0.006 (45/100): 0.006 (46/100): 0.005 (47/100): 0.008 (48/100): 0.009 (49/100): 0.000 (50/100): 0.008 (51/100): 0.008 (52/100): 0.002 (53/100): 0.011 (54/100): 0.008 (55/100): 0.006 (56/100): 0.004 (57/100): 0.012 (58/100): 0.000 (59/100): 0.008 (60/100): 0.007 (61/100): 0.008 (62/100): 0.009 (63/100): 0.000 (

In [8]:
y_h = np.array(corrections == batch[:,1])
same = batch[:,0] == batch[:,1]
not_corrected = np.array(corrections) == "False"
tn  = np.logical_and(same, not_corrected)
fn  = np.logical_and(np.logical_not(same), not_corrected)
fp  = np.logical_and(same, np.logical_not(not_corrected))
tp = np.logical_and(np.logical_not(same), np.logical_not(not_corrected))
ttp  = np.logical_and(tp, y_h)
TN = tn.sum(); FN = fn.sum(); FP = fp.sum(); TP = tp.sum(); TTP = ttp.sum(); TP = tp.sum(); FTP = TP-TTP
# [Total] True Positives, True True Positives, True Negatives, False Negatives, False Positives, False True Positives
f'TP={TP}, TTP={TTP}, TN={TN}, FN={FN}, FP={FP}, FTP={FTP}'

'TP=30, TTP=30, TN=50, FN=20, FP=0, FTP=0'

In [9]:
precision = TTP/(TP+FP)
recall    = TP/(TP+FN)
fpr       = FP/same.sum()
specificity = TN/(TN+FP+FTP)
accuracy    = (TTP+TN)/(TP+TN+FP+FN)
f1          = (2*precision*recall)/(precision+recall)
f'precision={precision:.3f}, recall={recall:.3f}, false positive rate={fpr:.3f}, specificity={specificity:.3f}, accuracy={accuracy:.3f}, f1={f1:.3f}'#, f'k={k}, a={ap}, b={bp}, log_exp={log_exp}, prob_exp={prob_exp}, relevency_t={relevency_t}, base_t={base_t}, threshold_e={threshold_e}'

'precision=1.000, recall=0.600, false positive rate=0.000, specificity=1.000, accuracy=0.800, f1=0.750'

In [10]:
corrections = []
for x, _ in batch: 
    corrections.append(blob_correct(x))

In [11]:
y_h = np.array(corrections == batch[:,1])
same = batch[:,0] == batch[:,1]
not_corrected = np.array(corrections) == batch[:,0]
tn  = np.logical_and(same, not_corrected)
fn  = np.logical_and(np.logical_not(same), not_corrected)
fp  = np.logical_and(same, np.logical_not(not_corrected))
tp = np.logical_and(np.logical_not(same), np.logical_not(not_corrected))
ttp  = np.logical_and(tp, y_h)
TN = tn.sum(); FN = fn.sum(); FP = fp.sum(); TP = tp.sum(); TTP = ttp.sum(); TP = tp.sum(); FTP = TP-TTP
# [Total] True Positives, True True Positives, True Negatives, False Negatives, False Positives, False True Positives
f'TP={TP}, TTP={TTP}, TN={TN}, FN={FN}, FP={FP}, FTP={FTP}'

'TP=27, TTP=2, TN=40, FN=23, FP=10, FTP=25'

In [12]:
precision = TTP/(TP+FP)
recall    = TP/(TP+FN)
fpr       = FP/same.sum()
specificity = TN/(TN+FP+FTP)
accuracy    = (TTP+TN)/(TP+TN+FP+FN)
f1          = (2*precision*recall)/(precision+recall)
f'precision={precision:.3f}, recall={recall:.3f}, false positive rate={fpr:.3f}, specificity={specificity:.3f}, accuracy={accuracy:.3f}, f1={f1:.3f}'#, f'k={k}, a={ap}, b={bp}, log_exp={log_exp}, prob_exp={prob_exp}, relevency_t={relevency_t}, base_t={base_t}, threshold_e={threshold_e}'

'precision=0.054, recall=0.540, false positive rate=0.200, specificity=0.533, accuracy=0.420, f1=0.098'